In [ ]:
import os
import sys
import numpy as np
import netCDF4 as nc
from tqdm.auto import tqdm # progress bar
import xarray as xr

import warnings
warnings.filterwarnings("ignore")

from ipynb.fs.full import Plots
from ipynb.fs.full import RMS
from ipynb.fs.full import Utils

import xesmf as xe

sys.path.append('ECCOv4-py/ECCOv4-py')
import ecco_v4_py as ecco

from config import input_dir, output_dir

11/14/24<br>
Produce 2 more sets of RMS plots for ECCO v4r5 and ctrl.
1.	Use the processed data I already have, where trends and MSC have been removed from each set of ECCO data.
2.	Divide the 2 sets of ECCO data (1992-2020) into 2 groups:
    - Data where the time step is in the GRACE data
    - Data where the time step in NOT in the GRACE data
3.	Plot RMS of v4r5, ctrl, and (v4r5 – ctrl) for each time set.


In [ ]:
# v4r5 and ctrl data
v4r5 = xr.open_dataset(os.path.join(input_dir, 'ecco_199201-201912-rm-means.nc'))
ctrl = xr.open_dataset(os.path.join(input_dir, 'pb_ECCO_ctrl.nc'))
v4r5 = Utils.convert_time(v4r5, 1992.0)
ctrl = Utils.convert_time(ctrl, 1992.0)

# this is the grace data after interpolated and aligned with ecco
aligned_grace = xr.open_dataset(os.path.join(input_dir, 'aligned_grace.nc'))
aligned_grace = aligned_grace.assign_coords({'time': aligned_grace['time'].astype('float32')})


In [ ]:
# Get weight of each cell (fraction of total ocean area in each cell)
weight = xr.open_dataset(os.path.join(input_dir, 'ecco_early_weight.nc'))
if 'dim_0' in weight.dims:
    weight = weight.rename({'dim_0': 'time', 'dim_1': 'tile', 'dim_2': 'j', 'dim_3': 'i', '__xarray_dataarray_variable__': 'pb'})
weight = Utils.convert_time(weight, 1992.0)

In [ ]:
def align_by_time_nans(ds, ds_base=aligned_grace):
    '''
    11/25/24
    Align ECCO data with GRACE times, return ds with nan wherever GRACE time data does not exist.

    '''
    base, ds_select_times = xr.align(ds_base, ds, join='inner')
    #ds_with_nans = ds.where(ds['time'] in ds_select_times['time'], np.nan)

    ds_with_nans = ds_select_times.reindex(time=ds.time)
    ds_with_nans = ds_with_nans.where(ds_with_nans != 0, np.nan)
    print('ds_with_nans')
    print(ds_with_nans['pb'].values)
    return ds_with_nans


In [ ]:
def align_by_time(ds, ds_base=aligned_grace):
    '''
    Align ECCO data with GRACE times, skip times that don't have GRACE data

    '''
    base, ds_select_times = xr.align(ds_base, ds, join='inner')
    #ds = ds.where(ds != 0, np.nan)
    print(f"time lengths, ds and selected times: {len(ds['time'])} {len(ds_select_times['time'])}")
    return ds_select_times

In [ ]:
def get_other_times_nans(ds, ds_base=aligned_grace):
    '''
    Get the data for all of the leftover times.
    Return ds with nan wherever GRACE data does exist.

    '''
    ds.assign_coords({'time': ds['time'].astype('float32')})
    ds_base.assign_coords({'time': ds_base['time'].astype('float32')})
    mask = ~(ds['time'].isin(ds_base['time']))  # Any time in ECCO, not in GRACE, gets marked as False.  Times in both are True.
    ds_other_times = ds.where(mask, drop=True)  # Drop all False times.

    ds_with_nans = ds_other_times.reindex(time=ds.time)
    ds_with_nans = ds_with_nans.where(ds_with_nans != 0, np.nan)

    return ds_with_nans

#ds1_unaligned = get_other_times(v4r5, aligned_grace)

In [ ]:
def get_other_times(ds, ds_base=aligned_grace):
    '''
    Get the data for all of the leftover times.
    Leave out the times for which there is no GRACE data.

    '''
    ds = ds.assign_coords({'time': ds['time'].astype('float32')})
    ds_base = ds_base.assign_coords({'time': ds_base['time'].astype('float32')})
    mask = ~(ds['time'].isin(ds_base['time']))  # Any time in ECCO, not in GRACE, gets marked as False.  Times in both are True.
    ds_other_times = ds.where(mask, drop=True)  # Drop all False times.
    return ds_other_times

#ds1_unaligned = get_other_times(v4r5, aligned_grace)

In [ ]:


def plot_rms_time_sets_nans(ds1, ds2, diff, names, titles, figure_name=None):
    '''
    Plot RMS of data1, data2, data1-data2

    '''
    ds1_aligned = align_by_time_nans(ds1)
    ds2_aligned = align_by_time_nans(ds2)
    diff_aligned = align_by_time_nans(diff)

    ds1_unaligned = get_other_times_nans(ds1, aligned_grace)
    ds2_unaligned = get_other_times_nans(ds2, aligned_grace)
    diff_unaligned = get_other_times_nans(diff, aligned_grace)

    aligned_names = [s + " overlaps GRACE" for s in names]
    unaligned_names = [s + " only ECCO" for s in names]
    
    RMS.rms_multi_time_plots([ds1_aligned, ds2_aligned, diff_aligned], \
                             weight, \
                             names + ['GRACE_times'], \
                             titles + [' Times that Align with GRACE'])
    RMS.rms_multi_time_plots([ds1_unaligned, ds2_unaligned, diff_unaligned], \
                              weight, \
                              names + ['NO_GRACE_times'], \
                              titles + [' Times Outside of GRACE'])
    RMS.rms_multi_time_plots([ds1_aligned, ds2_aligned, diff_aligned, ds1_unaligned, ds2_unaligned, diff_unaligned], \
                              weight, \
                              aligned_names + unaligned_names, \
                              titles + [' For All Times'],
                              figure_name = figure_name)


In [ ]:
# 12/2/24
# v4r5-ctrl_rmTrend_GRACE-rms-lat-lon-4.png  minus v4r5-ctrl_rmTrend_NO_GRACE-rms-lat-lon-4.png
# RMS.rms_plots(diff_aligned-diff_unaligned, weight, 'v4r5-ctrl_rmTrend_GRACE_minus_NO_GRACE', titles[0] + ' v4r5-ctrl Difference Between Times Coresponding to GRACE and Not', min=0, max=4)

# Run RMS over time and space, show plots
def rms_diff_plots(ds0, ds1, weight, name, title, min=None, max=None):
    
    rms_time_data_0 = RMS.rms_time(ds0, name, weight)
    rms_time_data_1 = RMS.rms_time(ds1, name, weight)

    fig = Plots.plot_rms_time(rms_time_data_0 - rms_time_data_1, title, ymin=min, ymax=max)
    fig.savefig(os.path.join(input_dir, f'{name}-rms-time.png'))
    del fig

    rms_space_data_0 = RMS.rms_space(ds0, name)
    rms_space_data_1 = RMS.rms_space(ds1, name)

    print()
    print(rms_space_data_0)
    print()
    print(rms_space_data_1)
    print()
    print(rms_space_data_0-rms_space_data_1)

    fig = Plots.plot_rms_world(rms_space_data_0-rms_space_data_1, title, -1, 1)
    # fig.savefig(os.path.join(input_dir, f'{name}-rms-lat-lon-{max}.png'))
    return fig

In [ ]:
def plot_rms_time_sets(ds1, ds2, diff, names, titles, figure_name=None):
    '''
    Plot RMS of data1, data2, data1-data2

    '''
    ds1_aligned = align_by_time(ds1)
    ds2_aligned = align_by_time(ds2)
    diff_aligned = align_by_time(diff)

    ds1_unaligned = get_other_times(ds1, aligned_grace)
    ds2_unaligned = get_other_times(ds2, aligned_grace)
    diff_unaligned = get_other_times(diff, aligned_grace)

    aligned_names = [s + " overlaps GRACE" for s in names]
    unaligned_names = [s + " only ECCO" for s in names]

    # v4r5-ctrl_rmTrend_GRACE-rms-lat-lon-4.png  minus v4r5-ctrl_rmTrend_NO_GRACE-rms-lat-lon-4.png
    return rms_diff_plots(diff_aligned, diff_unaligned, weight, names[3] + '_v4r5-ctrl_rmTrend_GRACE_minus_NO_GRACE', titles[0] + ' v4r5-ctrl Difference Between Times Coresponding to GRACE and Not', min=0, max=4)


    

In [ ]:
def plot_time_sets_v4r5_ctrl():
    names = ['v4r5', 'ctrl', 'v4r5-ctrl', 'rms']
    titles = ['RMS: ', 'V4R5', 'CTRL', 'V4R5-CTRL']
    from importlib import reload
    reload(Plots)
    plot_rms_time_sets(v4r5, ctrl, v4r5-ctrl, names, titles, 'v4r5_ctrl_all_times.png')

In [ ]:
# 11/25/24 plot with gaps for missing data instead of staight lines

def plot_gaps():

    names = ['v4r5', 'ctrl', 'v4r5-ctrl', 'all_times']
    titles = ['RMS: ', 'V4R5', 'CTRL', 'V4R5-CTRL']
    plot_rms_time_sets_nans(v4r5, ctrl, v4r5-ctrl, names, titles, 'v4r5_ctrl_all_times.png')